# Promovendo produtos em suas recomendações

Este notebook vai guiar você nas etapas de criação do filtro de promoção utilizando o [Amazon Personalize](https://aws.amazon.com/personalize/). Essa funcionalidade retorna recomendações que contenham resultados com produtos ou serviços promovidos. 

Recomendações personalizadas aumentam o engajamento de seu website ou aplicativo, e frequentemente existe a necessidade de aplicar regras adicionais de negócio no momento de apresentar itens para seus clientes. [O Filtro de Promoção do Amazon Personalize ](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/promoting-items.html) pode ajudar nos casos que é necessário mostrar itens de determinadas categorias ou marcas. Por exemplo, um aplicativo de video sob demanda pode promover uma nova série ou um website de e-commerce pode promover produtos que estão em liquidação.

![promotions-overview-pt-br.png](images/promotions-overview-pt-br.png "Diagrama demonstrando como o filtro de promoção muda o resultado da recomendação de produtos")

Primeiro precisamos seguir os passos para criar o grupo do conjunto de dados, em seguida criamos um recomendador que retornará produtos de um site de e-commerce ficticio. O objetivo é gerar recomendações que sejam relevantes para o usuário informado.

Depois, vamos criar o filtro de promoção para promover produtos que estarão presentes na lista de recomendações para o usuário.

Para finalizar, vamos fazer a limpeza dos recursos criados para evitar custos desnecessários.

O tempo estimado para executar todos os passos deste notebook é de aproximadamente 75 minutos.

## Como usar este notebook

O código está dividido em células como o exemplo abaixo. Você pode usar o botão triangular executar (Run) no topo desta página para executar cada célula e avançar para a seguinte. Ou dentro da célula você também pode usar a combinação de teclas "Shift + Enter" para a executar e avançar para a próxima célula.

Durante a execução do código na célula, observe uma linha que mostra um `*` enquanto a execução esta em andamento. Esta linha será atualizada para um número indicando que a execução do código desta célula finalizou.

Para começar, siga as instruções abaixo e execute as células do notebook.

## Introdução ao Amazon Personalize

[O Amazon Personalize](https://aws.amazon.com/pt/personalize//) simplifica o desenvolvimento de sistemas de recomendação, incluindo casos de recomendações de produtos em tempo real e marketing direto personalizado. O Amazon Personalize oferece a mesma tecnologia utilizada pela Amazon.com, sem necessidade de experiência prévia com aprendizado de máquina. Com o Amazon Personalize você paga pelo o que consome, sem taxas mínimas ou compromisso inicial. 

Para começar a utilizar o Amazon Personalize, basta seguir o processo com três passos que requer apenas alguns cliques na console AWS, ou algumas chamadas à API.

Primeiro, informe os dados de usuários, os dados do catálogo de produtos e dados de interações (como cliques, compras, etc). É possivel carregados os dados do Amazon S3 ou utilizar uma chamada de API.

Segundo, com um clique na console (ou uma chamada de API), treine o modelo de recomendação com os seus dados. Lembrando que todos os seus dados permanecem protegidos e seguros através de criptografia, e são utilizados somente para criar recomendações para seus cliente ou usuários.

Por último, receba recomendações personalizadas para qualquer usuário, criando um recomendador e utilizando a API GetRecommendations.

Caso queira conhecer mais sobre o Amazon Personalize, ou ter acesso a mais exemplos como este acesse: [Github Sample Notebooks](https://github.com/aws-samples/amazon-personalize-samples) e [Documentação do produto](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/what-is-personalize.html).

## Importação
O Python fornece uma coleção de bibliotecas e precisamos importá-las, como o [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para python) e [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que são ferramentas essenciais para a ciência de dados.

In [ ]:
# Instale a versão mais recente do botocore para garantir que temos os recursos mais atualizados no SDK
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

In [ ]:
# Importação de bibliotecas
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

In [ ]:
# Configure o SDK para o Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Utilize a célula abaixo para validar que o seu ambiente esta comunicando com Amazon Personalize.

In [ ]:
personalize.list_dataset_groups()

Informe o nome dos arquivos com os dados de itens e interações a serem utilizados.

In [ ]:
interactions_file_path = 'cleaned_interactions_training_data.csv'
items_file_path = 'cleaned_item_training_data.csv'

## Especifique um bucket S3 e uma localização de saída dos dados

O Amazon Personalize precisa de um bucket S3 como fonte de dados. O código abaixo irá criar um bucket com um `bucket_name` único.

O bucket do Amazon S3 precisa estar na mesma região que os recursos do Amazon Personalize.

In [ ]:
# Define a mesma região que o Notebook atual do Amazon SageMaker
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Ou você pode especificar a região onde o seu bucket e modelo serão utilizados
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("O bucket já existe. O nome será definido como: ", bucket_name)

## Download, preparação, e upload dos dados de treinamento

Produzimos dados sintéticos com base no código do projeto [Retail Demo Store] (https://github.com/aws-samples/retail-demo-store). Clique no link para maiores informações e possíveis usos.

Primeiro, faremos o download dos dados (dados de treinamento). Neste exemplo, utilizaremos o histórico de compras de um conjunto de dados de um e-commerce. O conjunto de dados contém o user_id, o item_id, e as interações entre dos clientes e itens, e também a hora em que essa interação ocorreu (Timestamp).

### Download e exploração do conjunto de dados de interações

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

O conjunto de dados foi transferido com sucesso com o nome: interactions.csv

Vamos explorar este conjunto de dados para identificar as suas caracteristicas.

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.info()

Das células acima, descobrimos que os nossos dados têm 5 colunas, 675004 linhas e os campos são: ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP e DISCOUNT.

Para que seja compativel com o esquema de interação do Amazon Personalize, as colunas precisam ter o nome esperado pelo Amazon Personalize (leia sobre nomes de coluna [aqui](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/how-it-works-dataset-schema.html) )

Os recomendadores ECOMMERCE requerem que dados do EVENT_TYPE sejam específicados para entender o contexto da interação. Vamos analisar os tipos de eventos que estão atualmente no nosso conjunto de dados:

In [ ]:
df.EVENT_TYPE.value_counts()

Verificamos que os eventos "View" e "Purchase" estão presentes. Eles serão utilizados em nosso exemplo.

### Preparação dos dados de interação


### Remova colunas

Algumas colunas deste conjunto de dados não adicionam valor ao nosso modelo e, por isso, devem ser removidas do conjunto de dados. Como exemplo a coluna com o nome *discount*.

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

Na célula abaixo, vamos salvar os dados limpos em um arquivo com o nome "final_training_data.csv

In [ ]:
df.to_csv(interactions_file_path)

### Download e exploração dos dados de itens

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .

O conjunto de dados foi transferido com sucesso com o nome items.csv

Vamos explorar este conjunto de dados para identificar as suas caracteristicas.

In [ ]:
items_df = pd.read_csv('./items.csv')
items_df

In [ ]:
items_df.info()

Vamos analisar os tipos de itens que estão incluídos no conjunto de dados.

In [ ]:
items_df.CATEGORY_L1.unique()

In [ ]:
items_df.CATEGORY_L2.unique()

### Remova colunas

Algumas colunas deste conjunto de dados poderiam acrescentar valor ao nosso modelo, mas não serão relevantes em nosso exemplo. Para simplificar, vamos remover a coluna com o nome *product_decription*.

In [ ]:
test=items_df.drop(columns=['PRODUCT_DESCRIPTION'])
items_df=test
items_df.sample(10)

Salve os dados limpos em um arquivo .csv

In [ ]:
items_df.to_csv(items_file_path)

## Configure o bucket S3 e funções IAM

Até agora, baixamos, limpamos e salvamos os dados em nosso notebook Jupyter utilizado. Mas o Amazon Personalize precisa de um bucket S3 para obter os dados, e também das permissões (função IAM) para acessar esse bucket. Vamos configurar isso na célula abaixo.

## Configure a politica do bucket S3
O Amazon Personalize precisa das pemissões necessárias para ler o conteúdo do seu bucket S3. Para isso adicione uma política de bucket.

Observação: verifique se a função que você está usando para executar o código deste notebook tem as permissões necessárias para modificar a política do bucket S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Upload dos dados de interação para o S3
Agora que os dados de treinamento estão preparados para o Amazon Personalize, precisamos carregá-los no bucket s3 criado acima

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path
    

### Upload dos dados de itens para S3
Agora que os dados de treinamento estão preparados para o Amazon Personalize, precisamos carregá-los no bucket s3 criado acima

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_file_path).upload_file(items_file_path)
items_s3DataPath = "s3://"+bucket_name+"/"+items_file_path

## Crie o grupo de conjunto de dados
O maior agrupamento no Personalize é um grupo de conjunto de dados, que isola seus dados, rastreadores de eventos, soluções, recomendadores e campanhas. Se preferir, pode alterar o nome abaixo.

Durante a criação é necessário informar o dominio dos dados, isso determina os esquemas e casos de utilização disponíveis para os recomendadores.

Para ter mais informações, acesse: [Documentação](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/data-prep-ds-group.html).

### Crie o grupo de conjunto de dados

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Antes de utilizarmos o grupo de conjunto de dados em qualquer etapa abaixo ele deve estar ativo (ACTIVE), execute a célula abaixo e aguarde que ele chegue nesse status para prosseguir.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 horas
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("Grupo de conjunto de dados: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Crie o esquema de interações
É através do esquema que o Amazon Personalize entende os seus dados. Esta configuração informa ao serviço como processar os dados do arquivo CSV. Observe que as colunas e tipos estão igual ao arquivo criado na etapa acima.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
        
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Crie o esquema de itens
É através do esquema que o Amazon Personalize entende os seus dados. Esta configuração informa ao serviço como processar os dados do arquivo CSV. Observe que as colunas e tipos estão igual ao arquivo criado na etapa acima.

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "PRICE",
            "type": "float"
        },
        {
            "name": "CATEGORY_L1",
            "type": ["string"],
            "categorical": True
        },
        {
            "name": "CATEGORY_L2",
            "type": ["string"],
            "categorical": True
            
        },
        {
            "name": "GENDER",
            "type": ["string"],
            "categorical": True
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-item_group",
    domain = "ECOMMERCE",
    schema = json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']

print(json.dumps(create_schema_response, indent=2))

## Crie o conjunto de dados
Após a criação do grupo, a próxima etapa será criar o conjunto de dados e depois carregá-los no Amazon Personalize.

### Crie o conjunto de dados de interação

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Crie o conjunto de dados de itens

In [ ]:
dataset_type = "ITEMS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Crie uma função IAM para o Personalize
O Amazon Personalize também precisa da acesso de assumir funções IAM na AWS para ter permissões de execução de determinadas tarefas.

Observação: verifique se a função utilizada para executar o código deste notebook tem as permissões necessárias para criar uma nova função IAM.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# O AmazonPersonalizeFullAccess permite o acesso a qualquer bucket S3 com um nome que inclua "personalize" ou " Personalize" 
# se quiser utilizar um bucket com um nome diferente, considere a possibilidade de criar e anexar uma nova política
# que forneça acesso de leitura ao seu bucket ou anexe a política AmazonS3ReadOnlyAccess à função
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Agora adicione o suporte ao S3
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # aguarde por 1 minuto para que a politica da função IAM se propague

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importe os dados
Anteriormente, foi criado o grupo e o conjunto de dados para armazenar os dados. A próxima tarefa será importar os dados que serão utilizados na construção do modelo.
### Crie a tarefa de importação do conjunto de dados de interações

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Crie a tarefa de importação do conjunto de dados de itens

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_items_import",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_file_path)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Aguarde para que a tarefa de importação fique com o status ACTIVE
Pode levar alguns minutos para a tarefa de importação seja concluída. Aguarde até que as tarefas fiquem com o status ativo (ACTIVE) na execução abaixo.

In [ ]:
max_time = time.time() + 3*60*60 # 3 horas

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("ConjuntodeDadosdeItensImport: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("ConjuntodeDadosdeInteracoesImport: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Escolha um recomendador

Cada domínio tem casos de uso distintos. Abaixo temos uma lista de receitas para o dominio de e-commerce.

In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # Veja a lista de receitas para o dominio ecommerce
display (available_recipes['recipes'])

Vamos criar um recomendador do tipo "Recomendado para você". Este recomendador oferece recomendações personalizadas para itens com base em um usuário especifico. Com este recomendador, o Amazon Personalize filtra automaticamente os itens que o usuário comprou com base no userId especificado e nos eventos `Purchase`.

[Informações adicionais para casos de dominio](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recomendado_para_voce_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Precisamos aguardar a criação do recomendador e fique com o status `ACTIVE`. Na célula abaixo verificamos o status a cada 60 segundos.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 horas
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]
    print(status)

    if status == "ACTIVE":
        print("Criação com sucesso para {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Criação falhou para  {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('A criação do recomendador "Recomendado para você" ainda esta em progresso')
        
    time.sleep(60)

## Obtendo recomendações
Agora que o recomendador foi treinado, vamos analisar as recomendações que conseguimos dar aos nossos usuários!

In [ ]:
# lendo os dados originais para ter um Dataframe que tenha ambos os item_ids 
# e os títulos correspondentes para facilitar a leitura das nossas recomendações.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    Este processo recebe um item_id de uma recomendação em formato de string, 
    converte-o para um int e, em seguida, faz uma pesquisa em um dataframe que retorna a descrição do item.
    
    Em caso de erro temos uma excessão criada.
    
    Fique a vontade para adicionar outros controles e debugs para melhorar o resultado caso tenha algum erro.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Erro ao obter a descrição do item"

In [ ]:
def get_category_by_id(item_id, item_df):
    """
    Este processo recebe um item_id de uma recomendação em formato de string, 
    converte-o para um int e, em seguida, faz uma pesquisa em um dataframe e retorna a descrição do item.
    
    Em caso de erro temos uma excessão criada.
    """
    
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['CATEGORY_L2'].values[0]
    except:
        print (item_id)
        return "Erro ao obter a categoria do item"
    

Vamos receber algumas recomendações do recomendador "Recomendado para você":

In [ ]:
# Primeiro selecione um usuário
test_user_id = "777" 

# Solicite a recomentação para o usuário
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Crie um novo dataframe para as recomendações
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

## Utilizando filtro de promoção com um recomendador
Vamos criar um filtro de promoção para garantir que as recomendações para os usuários contêm itens que queremos promover. A nossa loja de e-commerce está promovendo produtos para o Halloween, e queremos garantir que os usuários vejam produtos desta categoria.

Para maiores informações sobre filtro de promoção, acesse: [documentação](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/promoting-items.html). 

### Crie um filtro para promover itens

Primeiro, precisamos de criar um filtro que defina quais os produtos que serão promovidos. Utilizaremos um filtro dinâmico [filtro](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/filter-expressions.html) que recebe um valor em tempo de inferência. 

Este filtro inclui apenas os itens que têm o valor especificado para a coluna *Items.CATEGORY_L2*.

Para obter os mesmos resultados, também poderíamos utilizar um filtro estático da seguinte forma:

```
'INCLUDE ItemID WHERE Items.CATEGORY_L2 IN ("halloween")'
```

Com o uso do filtro dinâmico temos maior flexibilidade, e caso seja necessário promover produtos de uma outra categoria não será necessário criar um novo filtro.

In [ ]:
create_filter_response = personalize.create_filter(
    name = 'filtro_categoria',
    datasetGroupArn = dataset_group_arn,
    filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L2 IN ($CATEGORY)'
) 
filter_arn = create_filter_response["filterArn"]
print("Filter ARN: " + filter_arn)

Aguarde até que o filtro  criado fique com o status: "Active".

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 horas
    
while time.time() < max_time:
    version_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = version_response["filter"]["status"]

    if status == "ACTIVE":
        print("Criação concluída para {}".format(filter_arn))
        
    elif status == "CREATE FAILED":
        print("Criação falhou para {}".format(filter_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('A criação do filtro está em progresso')
        
    time.sleep(30)

### Receba recomentações para o filtro criado

Agora que o filtro foi criado, podemos receber recomendações para um usuário utilizando o filtro. Vamos usar o usuário de teste previamente configurado.

In [ ]:
print(test_user_id)

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20,
    filterArn = filter_arn,
    filterValues={"CATEGORY" : "\"halloween\""}
)
user_recommendations_df =[]

# Criar um novo dataframe para as recomendações
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

Podemos observar que todos os itens retornados têm "category l2" como "halloween".

### Recebendo recomendações com o uso do filtro de promoção

Queremos fazer recomendações personalizadas para cada usuário, mas em vez de ter todos os itens de uma determinada categoria, queremos ainda incluir alguns itens para a promover a categoria "Halloween".  Para isso vamos usar o filtro de promoção!

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20,
    promotions = [{
        "name" : "halloween_promotion",
        "percentPromotedItems" : 20,
        "filterArn": filter_arn,
        "filterValues": {
            "CATEGORY" : "\"halloween\""
        }
    }]
)


# Criar um novo dataframe para as recomendações
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns = ["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows =20
display(user_recommendations_df)

### Promovendo produtos enquanto aplicamos o filtro

É possível utilizar um filtro combinado com a ação de promover produtos. O filtro no nível superior do bloco de parâmetros é aplicado apenas aos itens não promovidos. O filtro para selecionar os produtos promovidos é especificado no bloco de parâmetros `promotions`. O exemplo a seguir utiliza o mesmo filtro dinâmico utilizado anteriormente. O primeiro é aplicado aos produtos não promovidos, selecionando os produtos do nível 2 da categoria "decorative", e o segundo é aplicado ao filtro de promoção, promovendo produtos do nível 2 da categoria "halloween".

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn=recommended_for_you_arn,
    userId=test_user_id,
    numResults=20,
    filterArn=filter_arn,
    filterValues={
        "CATEGORY": "\"decorative\""
    },
    promotions=[{
        "name": "halloween_promotion",
        "percentPromotedItems": 20,
        "filterArn": filter_arn,
        "filterValues": {
            "CATEGORY": "\"halloween\""
        }
    }]
)

# Criar um novo dataframe para as recomendações
item_list = get_recommendations_response['itemList']
recommendation_id_list = []
recommendation_description_list = []
recommendation_category_list = []

for item in item_list:
    description = get_item_by_id(item['itemId'], items_df)
    recommendation_description_list.append(description)
    recommendation_id_list.append(item['itemId'])
    recommendation_category_list.append(
        get_category_by_id(item['itemId'], items_df))

user_recommendations_df = pd.DataFrame(recommendation_id_list, columns=["ID"])
user_recommendations_df["description"] = recommendation_description_list
user_recommendations_df["category level 2"] = recommendation_category_list

pd.options.display.max_rows = 20
display(user_recommendations_df)

## Conclusão
Usando este notebook, foi possível treinar com sucesso um modelo de aprendizado de máquina (Machine Learning) para gerar recomendações de produtos com base no comportamento passado do usuário. Você criou um recomendador para um cenário de uso básico e usou filtros e [Filtro de Promoção](https://docs.aws.amazon.com/pt_br/personalize/latest/dg/promoting-items.html) para aplicar regras de negócio adicionais aos produtos que devem ser apresentados aos usuários.

Como próximo passo, é possível adaptar este código para criar outros recomendadores de acordo com as suas necessidades.

Após concluir este exemplo, certifique-se de seguir os passos da próxima sessão para remover os recursos criados neste notebook.

## Remoção dos recursos
Esta seção fornece instruções sobre como remover os recursos criados neste notebook

### Salve as informações do recurso para a remoção:
Salve a informação dos recursos criados para utilizar no próximo notebook `Limpeza_de_Recursos.ipynb`.

Importante: Isto substituirá qualquer dado armazenado para estas variáveis e as definirá com os valores neste notebook.

In [ ]:
# Salve as informações dos recursos criados para posterior limpeza
%store dataset_group_arn
%store role_name
%store region
print ('dataset_group_arn:', dataset_group_arn)
print ('role_name:', role_name)
print ('region:', region)

### Execute o notebook de remoção de recursos

Prossiga para [Limpeza_de_Recursos.ipynb](Limpeza_de_Recursos.ipynb) limpar os recursos.